In [ ]:
import os
import sys

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
try:
    import open3d as o3d
except ImportError:
    print(sys.exc_info())
import pandas as pd
from scipy import interpolate
import quadpy

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.integrate import elementwise_quad, elementwise_dblquad
from dosipy.utils.viz import (set_colorblind, scatter_2d, scatter_3d,
                              fig_config, colormap_from_array)
sys.path.append('..')
from helpers import (clean_df, export_pcd, export_fields,
                       poynting_vector, get_imcolors, export_rect_idx)

In [ ]:
# set input data

polarization = 'te'
f = 26

if f == 26:
    edge_length = 20
else:
    edge_length = 10
area = edge_length ** 2

# COMSOL

In [ ]:
fname = f'APDn_{f}GHz_{polarization.upper()}_{int(area/100)}cm2.txt'
data = np.loadtxt(os.path.join('COMSOL', fname), comments='%')
xyz = data[:, :3] * 1000
APDn = data[:, -1]

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
pcd.colors = o3d.utility.Vector3dVector(colormap_from_array(-APDn))
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([-10, -10, -20])
)
#o3d.visualization.draw_geometries([pcd, cframe])

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)

diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 10 ** 4
camera = [center[0] - diameter, center[1], center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_zy = xyz[mask]
APD_zy = np.abs(APDn[mask])

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_zy)
center = pcd.get_center()
pcd.colors = o3d.utility.Vector3dVector(colormap_from_array(APD_zy))
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([-10, -10, -20])
)
#o3d.visualization.draw_geometries([pcd, cframe])

In [ ]:
set_colorblind()
label = '$APD$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_2d({'$y$ [mm]': xyz_zy[:, 1],
                      '$z$ [mm]': xyz_zy[:, 2],
                      label: APD_zy}, s=1)

In [ ]:
degree = 21
points = np.c_[xyz_zy[:, 1], xyz_zy[:, 2]]
bbox = [points[:, 0].min(), points[:, 0].max(), points[:, 1].min(), points[:, 1].max()]

In [ ]:
func = interpolate.LinearNDInterpolator(points, APD_zy, fill_value=1)

In [ ]:
set_colorblind()
label = '$APD$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_2d({'$y$ [mm]': xyz_zy[:, 1],
                      '$z$ [mm]': xyz_zy[:, 2],
                      label: func(points)}, s=1)

In [ ]:
scheme = quadpy.c2.get_good_scheme(degree)
val = scheme.integrate(lambda x: func(x[0], x[1]),
                       quadpy.c2.rectangle_points([points[:, 0].min(), points[:, 0].max()],
                                                  [points[:, 1].min(), points[:, 1].max()]))
val / 400

In [ ]:
val = elementwise_dblquad(points, APD_zy, degree=degree,
                          interp_func=interpolate.CloughTocher2DInterpolator, fill_value=0)
val / 400

# DosiPy

In [ ]:
# load surface data

if polarization == 'te':
    origin = [-25.28, 17.25]
else:
    origin = [-19.77, 5.22]

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([6, -25, -20])
)
#o3d.visualization.draw_geometries([pcd, cframe])

In [ ]:
# postprocessing on the surface of the model

E, H = export_fields(df)
Sx, Sy, Sz = poynting_vector(E, H)

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)

diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 10 ** 5
camera = [center[0] + diameter, center[1] * 2.5, center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_zy = xyz[mask]

In [ ]:
# compute averaged APD

avg_center = [origin[0]+edge_length/2,
              origin[1]+edge_length/2]
area = edge_length ** 2
pAPD_origin, idx_rect = export_rect_idx(xyz=xyz_zy,
                                        center=avg_center,
                                        edge_length=edge_length,
                                        view='zy')
xyz_rect = xyz_zy[idx_rect]

In [ ]:
pcd_rect = o3d.geometry.PointCloud()
pcd_rect.points = o3d.utility.Vector3dVector(xyz_rect)
pcd_rect.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(30))
pcd_rect.orient_normals_consistent_tangent_plane(30)
#o3d.visualization.draw_geometries([pcd_rect], point_show_normal=True)

In [ ]:
n = np.asarray(pcd_rect.normals)

APD_rect = np.abs(Sx[mask][idx_rect].real * n[:, 0]
                  + Sy[mask][idx_rect].real * n[:, 1]
                  + Sz[mask][idx_rect].real * n[:, 2])

In [ ]:
set_colorblind()
label = '$APD$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_2d({'$y$ [mm]': xyz_rect[:, 1],
                      '$z$ [mm]': xyz_rect[:, 2],
                      label: APD_rect}, s=1)

In [ ]:
degree = 21
points = np.c_[xyz_rect[:, 2], xyz_rect[:, 1]]
bbox = [origin[0], origin[0]+edge_length, origin[1], origin[1]+edge_length]

In [ ]:
func = interpolate.SmoothBivariateSpline(points[:, 0], points[:, 1], APD_rect, bbox=bbox, kx=3, ky=3)

scheme = quadpy.c2.get_good_scheme(degree)
val = scheme.integrate(lambda x: func(x[0], x[1], grid=False),
                       quadpy.c2.rectangle_points([origin[0], origin[0]+edge_length],
                                                  [origin[1], origin[1]+edge_length]))
APD_av = val / area
APD_av

In [ ]:
func = interpolate.CloughTocher2DInterpolator(points, APD_rect, fill_value=0)

scheme = quadpy.c2.get_good_scheme(degree)
val = scheme.integrate(lambda x: func(x[0], x[1]),
                       quadpy.c2.rectangle_points([origin[0], origin[0]+edge_length],
                                                  [origin[1], origin[1]+edge_length]))
APD_av = val / area
APD_av

In [ ]:
func = interpolate.NearestNDInterpolator(points, APD_rect)

scheme = quadpy.c2.get_good_scheme(degree)
val = scheme.integrate(lambda x: func(x[0], x[1]),
                       quadpy.c2.rectangle_points([origin[0], origin[0]+edge_length],
                                                  [origin[1], origin[1]+edge_length]))
APD_av = val / area
APD_av